### Neural Network OOP code
1.  Object-Oriented Structure:
2.  A NeuralNetwork Model class handles model training and prediction.
3.  It can be easily reused, extended, or swapped for another model.

A prediction is made using new data.



In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

class NeuralNetworkModel:
    def __init__(self, hidden_layer_sizes=(10,), max_iter=1000, random_state=42):
        """Initializes the neural network model."""
        self.model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                                   max_iter=max_iter,
                                   random_state=random_state)

    def train(self, X_train, y_train):
        """Trains the neural network with training data."""
        self.model.fit(X_train, y_train)

    def predict(self, X_test):
        """Makes predictions on new data."""
        return self.model.predict(X_test)

    def evaluate(self, X_test, y_test):
        """Evaluates the model's performance."""
        predictions = self.predict(X_test)
        return accuracy_score(y_test, predictions)


# Example usage with Iris dataset
if __name__ == "__main__":
    # Load sample data
    iris = load_iris()
    X, y = iris.data, iris.target

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and train the model
    nn = NeuralNetworkModel(hidden_layer_sizes=(5, 5))
    nn.train(X_train, y_train)

    # Evaluate and make a prediction
    accuracy = nn.evaluate(X_test, y_test)
    print(f"Model accuracy: {accuracy:.2f}")

    sample = X_test[0].reshape(1, -1)
    prediction = nn.predict(sample)
    print(f"Prediction for sample {sample}: {iris.target_names[prediction[0]]}")


Model accuracy: 0.90
Prediction for sample [[6.1 2.8 4.7 1.2]]: virginica


/home/codespace/.local/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


### Second Example

In [6]:
import numpy as np

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        # Initialize weights and biases
        self.learning_rate = learning_rate
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.bias_hidden = np.zeros((1, hidden_size))
        
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_output = np.zeros((1, output_size))
    
    def forward(self, X):
        # Feedforward step
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = sigmoid(self.final_input)
        return self.final_output

    def backward(self, X, y, output):
        # Calculate error
        error = y - output
        
        # Calculate gradients
        d_output = error * sigmoid_derivative(output)
        error_hidden = d_output.dot(self.weights_hidden_output.T)
        d_hidden = error_hidden * sigmoid_derivative(self.hidden_output)
        
        # Update weights and biases
        self.weights_hidden_output += self.hidden_output.T.dot(d_output) * self.learning_rate
        self.bias_output += np.sum(d_output, axis=0, keepdims=True) * self.learning_rate
        
        self.weights_input_hidden += X.T.dot(d_hidden) * self.learning_rate
        self.bias_hidden += np.sum(d_hidden, axis=0, keepdims=True) * self.learning_rate

    def train(self, X, y, epochs=10000):
        for i in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)
            if i % 1000 == 0:
                loss = np.mean((y - output) ** 2)
                print(f"Epoch {i}, Loss: {loss:.4f}")
    
    def predict(self, X):
        output = self.forward(X)
        return np.round(output)

# Example: XOR problem
if __name__ == "__main__":
    # Input data (XOR)
    X = np.array([[0, 0],
                  [0, 1],
                  [1, 0],
                  [1, 1]])
    
    # Labels
    y = np.array([[0],
                  [1],
                  [1],
                  [0]])
    
    # Create and train the model
    model = SimpleNeuralNetwork(input_size=2, hidden_size=4, output_size=1)
    model.train(X, y, epochs=10000)

    # Predict
    predictions = model.predict(X)
    print("Predictions:")
    print(predictions)


Epoch 0, Loss: 0.3098
Epoch 1000, Loss: 0.2077
Epoch 2000, Loss: 0.1236
Epoch 3000, Loss: 0.0380
Epoch 4000, Loss: 0.0145
Epoch 5000, Loss: 0.0079
Epoch 6000, Loss: 0.0052
Epoch 7000, Loss: 0.0038
Epoch 8000, Loss: 0.0030
Epoch 9000, Loss: 0.0024
Predictions:
[[0.]
 [1.]
 [1.]
 [0.]]


## Third example

#### A NeuralNetworkModel class handles model training and prediction.

#### It can be easily reused, extended, or swapped for another model.

#### A prediction is made using new data.

In [7]:
import numpy as np

class Layer:
    def __init__(self, input_size, output_size):
        # Initialize weights and biases
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.biases = np.zeros((1, output_size))
        
    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.biases
        return self.activation(self.z)
    
    def activation(self, z):
        # Sigmoid activation function
        return 1 / (1 + np.exp(-z))
    
    def backward(self, output_gradient, learning_rate):
        # Calculate gradients
        activation_gradient = self.z * (1 - self.z)  # Derivative of sigmoid
        weighted_gradient = output_gradient * activation_gradient
        self.weights_gradient = np.dot(self.inputs.T, weighted_gradient)
        self.biases_gradient = np.sum(weighted_gradient, axis=0, keepdims=True)
        
        # Update weights and biases
        self.weights -= learning_rate * self.weights_gradient
        self.biases -= learning_rate * self.biases_gradient
        
        # Return the gradient for the previous layer
        return np.dot(weighted_gradient, self.weights.T)


class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        
    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs
    
    def backward(self, output_gradient, learning_rate):
        for layer in reversed(self.layers):
            output_gradient = layer.backward(output_gradient, learning_rate)
    
    def train(self, x_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            # Forward pass
            output = self.forward(x_train)
            # Compute loss (Mean Squared Error)
            loss = np.mean((output - y_train) ** 2)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss}")
            # Backward pass
            output_gradient = 2 * (output - y_train) / y_train.size
            self.backward(output_gradient, learning_rate)


# Example usage
if __name__ == "__main__":
    # Sample data (XOR problem)
    x_train = np.array([[0, 0],
                        [0, 1],
                        [1, 0],
                        [1, 1]])
    
    y_train = np.array([[0], [1], [1], [0]])  # XOR output
    
    # Create a simple neural network with one hidden layer
    layers = [
        Layer(input_size=2, output_size=2),  # Hidden layer
        Layer(input_size=2, output_size=1)   # Output layer
    ]
    
    nn = NeuralNetwork(layers)
    nn.train(x_train, y_train, epochs=10000, learning_rate=0.1)

Epoch 1/10000, Loss: 0.2500017617052654
Epoch 2/10000, Loss: 0.25000176030641463
Epoch 3/10000, Loss: 0.250001758909226
Epoch 4/10000, Loss: 0.2500017575136967
Epoch 5/10000, Loss: 0.2500017561198242
Epoch 6/10000, Loss: 0.25000175472760594
Epoch 7/10000, Loss: 0.2500017533370392
Epoch 8/10000, Loss: 0.25000175194812135
Epoch 9/10000, Loss: 0.2500017505608499
Epoch 10/10000, Loss: 0.2500017491752223
Epoch 11/10000, Loss: 0.25000174779123574
Epoch 12/10000, Loss: 0.25000174640888784
Epoch 13/10000, Loss: 0.2500017450281759
Epoch 14/10000, Loss: 0.25000174364909733
Epoch 15/10000, Loss: 0.2500017422716496
Epoch 16/10000, Loss: 0.25000174089583016
Epoch 17/10000, Loss: 0.2500017395216365
Epoch 18/10000, Loss: 0.25000173814906596
Epoch 19/10000, Loss: 0.250001736778116
Epoch 20/10000, Loss: 0.2500017354087841
Epoch 21/10000, Loss: 0.2500017340410678
Epoch 22/10000, Loss: 0.2500017326749644
Epoch 23/10000, Loss: 0.25000173131047143
Epoch 24/10000, Loss: 0.2500017299475864
Epoch 25/10000, Lo

### Fourth Example

In [8]:
import numpy as np

class Layer:
    def __init__(self, input_size, output_size):
        # Initialize weights and biases
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.biases = np.zeros((1, output_size))
        
    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.biases
        return self.activation(self.z)
    
    def activation(self, z):
        # Sigmoid activation function
        return 1 / (1 + np.exp(-z))
    
    def backward(self, output_gradient, learning_rate):
        # Calculate gradients
        activation_gradient = self.z * (1 - self.z)  # Derivative of sigmoid
        weighted_gradient = output_gradient * activation_gradient
        self.weights_gradient = np.dot(self.inputs.T, weighted_gradient)
        self.biases_gradient = np.sum(weighted_gradient, axis=0, keepdims=True)
        
        # Update weights and biases
        self.weights -= learning_rate * self.weights_gradient
        self.biases -= learning_rate * self.biases_gradient
        
        # Return the gradient for the previous layer
        return np.dot(weighted_gradient, self.weights.T)


class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        
    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs
    
    def backward(self, output_gradient, learning_rate):
        for layer in reversed(self.layers):
            output_gradient = layer.backward(output_gradient, learning_rate)
    
    def train(self, x_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            # Forward pass
            output = self.forward(x_train)
            # Compute loss (Mean Squared Error)
            loss = np.mean((output - y_train) ** 2)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss}")
            # Backward pass
            output_gradient = 2 * (output - y_train) / y_train.size
            self.backward(output_gradient, learning_rate)


# Example usage
if __name__ == "__main__":
    # Sample data (XOR problem)
    x_train = np.array([[0, 0],
                        [0, 1],
                        [1, 0],
                        [1, 1]])
    
    y_train = np.array([[0], [1], [1], [0]])  # XOR output
    
    # Create a simple neural network with one hidden layer
    layers = [
        Layer(input_size=2, output_size=2),  # Hidden layer
        Layer(input_size=2, output_size=1)   # Output layer
    ]
    
    nn = NeuralNetwork(layers)
    nn.train(x_train, y_train, epochs=10000, learning_rate=0.1)

Epoch 1/10000, Loss: 0.25000804070192223
Epoch 2/10000, Loss: 0.25000802720788307
Epoch 3/10000, Loss: 0.25000801374766013
Epoch 4/10000, Loss: 0.2500080003211408
Epoch 5/10000, Loss: 0.2500079869282129
Epoch 6/10000, Loss: 0.2500079735687649
Epoch 7/10000, Loss: 0.25000796024268557
Epoch 8/10000, Loss: 0.2500079469498641
Epoch 9/10000, Loss: 0.25000793369019025
Epoch 10/10000, Loss: 0.25000792046355425
Epoch 11/10000, Loss: 0.2500079072698467
Epoch 12/10000, Loss: 0.2500078941089585
Epoch 13/10000, Loss: 0.2500078809807814
Epoch 14/10000, Loss: 0.25000786788520746
Epoch 15/10000, Loss: 0.25000785482212884
Epoch 16/10000, Loss: 0.2500078417914385
Epoch 17/10000, Loss: 0.2500078287930298
Epoch 18/10000, Loss: 0.25000781582679643
Epoch 19/10000, Loss: 0.2500078028926326
Epoch 20/10000, Loss: 0.2500077899904328
Epoch 21/10000, Loss: 0.25000777712009237
Epoch 22/10000, Loss: 0.2500077642815064
Epoch 23/10000, Loss: 0.2500077514745711
Epoch 24/10000, Loss: 0.25000773869918264
Epoch 25/10000